In [203]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets

In [204]:
#zaimportowanie pliku z danymi
df = pd.read_csv("../WA_Fn-UseC_-HR-Employee-Attrition.csv") 
#df

In [205]:
# wybranie przydzielonych kolumn + Attrition
df_ds = df[["Attrition", "TrainingTimesLastYear", "TotalWorkingYears",
       "YearsAtCompany", "YearsInCurrentRole", "YearsSinceLastPromotion",
       "YearsWithCurrManager", "StockOptionLevel"]] 
df_ds

,Attrition,TrainingTimesLastYear,TotalWorkingYears,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,StockOptionLevel
0,Yes,0,8,6,4,0,5,0
1,No,3,10,10,7,1,7,1
2,Yes,3,7,0,0,0,0,0
3,No,3,8,8,7,3,0,0
4,No,3,6,2,2,2,2,1
...,...,...,...,...,...,...,...,...
1465,No,3,17,5,2,0,3,1
1466,No,5,9,7,7,1,7,1
1467,No,0,6,6,2,0,3,1
1468,No,3,17,9,6,0,8,0


# TrainingTimesLastYear vs. Attrition #

In [206]:
# Tabela przestawna dla liczby szkoleń w ostatnim roku
TTLY = pd.crosstab(df_ds["Attrition"], df_ds["TrainingTimesLastYear"]) 
TTLY

TrainingTimesLastYear,0,1,2,3,4,5,6
Attrition,,,,,,,
No,39,62,449,422,97,105,59
Yes,15,9,98,69,26,14,6


In [207]:
# Number of employees that left a company vs. the number of trainings in the last year
# Create a copy of dataframe with Attrition=Yes
df_ds_TTLY_YES = df_ds[df_ds["Attrition"]=="Yes"].copy()
# Count the employees that left the company
df_ds_TTLY_YES_only = df_ds_TTLY_YES.groupby(by="TrainingTimesLastYear").count() ["Attrition"].reset_index()
# Change the name of column Attrition -> Atttrition_YES
df_ds_TTLY_YES_only = df_ds_TTLY_YES_only.rename(columns = {"Attrition":"Attrition_YES"})
# df_ds_TTLY_YES_only

In [208]:
# Number of employees that stayed in a company vs. the number of trainings in the last year
# Create a copy of dataframe with Attrition=No
df_ds_TTLY_NO = df_ds[df_ds["Attrition"]=="No"].copy()
# Count the employees that stayed in a company
df_ds_TTLY_NO_only = df_ds_TTLY_NO.groupby(by="TrainingTimesLastYear").count() ["Attrition"].reset_index()
# Change the name of column Attrition -> Atttrition_NO
df_ds_TTLY_NO_only = df_ds_TTLY_NO_only.rename(columns = {"Attrition":"Attrition_NO"})
# df_ds_TTLY_NO_only

In [209]:
# Merge tables with trainings and attrition both YES and NO
df_ds_TTLY_Y_N = pd.merge(df_ds_TTLY_YES_only, df_ds_TTLY_NO_only, on="TrainingTimesLastYear", how="left")
# Reset index
df_ds_TTLY_Y_N.reset_index()
# Add a column with total number of employees
df_ds_TTLY_Y_N["Yes+No"] = df_ds_TTLY_Y_N["Attrition_YES"] + df_ds_TTLY_Y_N["Attrition_NO"]
# Add a column with probability of leaving a company
df_ds_TTLY_Y_N["Probability_YES"] = df_ds_TTLY_Y_N["Attrition_YES"]/df_ds_TTLY_Y_N["Yes+No"]
print(df_ds_TTLY_Y_N)

   TrainingTimesLastYear  Attrition_YES  Attrition_NO  Yes+No  Probability_YES
0  0                      15             39            54      0.277778       
1  1                      9              62            71      0.126761       
2  2                      98             449           547     0.179159       
3  3                      69             422           491     0.140530       
4  4                      26             97            123     0.211382       
5  5                      14             105           119     0.117647       
6  6                      6              59            65      0.092308       


In [210]:
# Characteristics of slider for number of trainings in the last year

from IPython.display import display
from ipywidgets import HBox, Label

TTLY_slider = widgets.IntSlider(
    value=0,
    min=0,
    max=6,
    step=1,
    description='',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

# display(TTLY_slider)
HBox([Label('Liczba szkoleń w ostatnim roku:'), TTLY_slider])

In [211]:
# Slider connected with value_handler function returning probability
def value_handler(v):
    TrainNo = TTLY_slider.value
    Attr_Y_prob = df_ds_TTLY_Y_N.loc[df_ds_TTLY_Y_N["TrainingTimesLastYear"]==TrainNo, "Probability_YES"]
    print("Prawdopodobieństwo odejścia z firmy wynosi: {}".format(Attr_Y_prob[TrainNo]))


widgets.interact(value_handler, v=TTLY_slider)

interactive(children=(IntSlider(value=0, continuous_update=False, description='v', max=6), Output()), _dom_cla…

<function __main__.value_handler(v)>

# YearsAtCompany vs. Attrition #

In [212]:
YAC = pd.crosstab(df_ds["Attrition"], df_ds["YearsAtCompany"]) # Tabela przestawna dla lat przepracowanych w firmie
YAC

YearsAtCompany,0,1,2,3,4,5,...,32,33,34,36,37,40
Attrition,,,,,,,,,,,,,
No,28,112,100,108,91,175,...,2,4,1,2,1,0
Yes,16,59,27,20,19,21,...,1,1,0,0,0,1


In [213]:
df_ds["YearsAtCompany"].max()

40

In [214]:
print(df_ds.groupby(by=["YearsAtCompany", "Attrition"]).count()["StockOptionLevel"])

YearsAtCompany  Attrition
0               No           28 
                Yes          16 
1               No           112
                Yes          59 
2               No           100
                            ... 
33              Yes          1  
34              No           1  
36              No           2  
37              No           1  
40              Yes          1  
Name: StockOptionLevel, Length: 64, dtype: int64


In [215]:
# Create copy of df_ds with Attrition and YearsAtCompany only
df_ds_YAC_YES = df_ds.loc[df_ds["Attrition"]=="Yes", ["Attrition", "YearsAtCompany"]].copy()
# Gather data in bins to avoid empty values
df_ds_YAC_YES["YearsAtCompany_bins"] = pd.cut(df_ds_YAC_YES["YearsAtCompany"], bins=[-1,3,7,10,30])
df_ds_YAC_YES["YearsAtCompany_bins"].value_counts()

(-1, 3]     122
(3, 7]      60 
(7, 10]     35 
(10, 30]    16 
Name: YearsAtCompany_bins, dtype: int64

In [216]:
# Number of employees that left a company vs. the number of years worked in a company
# Create a copy of dataframe with Attrition=Yes
df_ds_YAC_YES = df_ds[df_ds["Attrition"]=="Yes"].copy()
df_ds_YAC_YES["YearsAtCompany_bins"] = pd.cut(df_ds_YAC_YES["YearsAtCompany"], bins=[-1,3,7,10,30])
# Count the employees that left the company
df_ds_YAC_YES_only = df_ds_YAC_YES.groupby(by="YearsAtCompany_bins").count() ["Attrition"].reset_index()
# Change the name of column Attrition -> Atttrition_YES
df_ds_YAC_YES_only = df_ds_YAC_YES_only.rename(columns = {"Attrition":"Attrition_YES"})
# df_ds_YAC_YES_only

In [217]:
# Number of employees that stayed in a company vs. the number of years worked in a company
# Create a copy of dataframe with Attrition=No
df_ds_YAC_NO = df_ds[df_ds["Attrition"]=="No"].copy()
df_ds_YAC_NO["YearsAtCompany_bins"] = pd.cut(df_ds_YAC_NO["YearsAtCompany"], bins=[-1,3,7,10,30])
# Count the employees that stayed in a company
df_ds_YAC_NO_only = df_ds_YAC_NO.groupby(by="YearsAtCompany_bins").count() ["Attrition"].reset_index()
# Change the name of column Attrition -> Atttrition_NO
df_ds_YAC_NO_only = df_ds_YAC_NO_only.rename(columns = {"Attrition":"Attrition_NO"})
# df_ds_YAC_NO_only

In [218]:
# Merge tables with trainings and attrition both YES and NO
df_ds_YAC_Y_N = pd.merge(df_ds_YAC_YES_only, df_ds_YAC_NO_only, on="YearsAtCompany_bins", how="left")
# Reset index
df_ds_YAC_Y_N.reset_index()
# Add a column with total number of employees
df_ds_YAC_Y_N["Yes+No"] = df_ds_YAC_Y_N["Attrition_YES"] + df_ds_YAC_Y_N["Attrition_NO"]
# Add a column with probability of leaving a company
df_ds_YAC_Y_N["Probability_YES"] = df_ds_YAC_Y_N["Attrition_YES"]/df_ds_YAC_Y_N["Yes+No"]
df_ds_YAC_Y_N

,YearsAtCompany_bins,Attrition_YES,Attrition_NO,Yes+No,Probability_YES
0,"(-1, 3]",122,348,470,0.259574
1,"(3, 7]",60,412,472,0.127119
2,"(7, 10]",35,247,282,0.124113
3,"(10, 30]",16,214,230,0.069565


In [219]:
# Characteristics of slider for the number of years worked in a company
from IPython.display import display
from ipywidgets import HBox, Label

YAC_toggle = widgets.ToggleButtons(
    options=['(0-3>', '(3-7>', '(7-10>', '(10-30)'],
    description='',
    disabled=False,
    button_style=''
)

# display(YAC_slider)
HBox([Label('Liczba lat przepracowanych w firmie:'), YAC_toggle])

In [220]:
# Nie mogłem się odwołać do tego pogrupowanego indexu, więc zamieniłem na wartości z indexu podstawowego
def value_handler(v):
    if YAC_toggle.value == "(0-3>":
        CYearsNo = 0 #"(-1, 3]"
    elif YAC_toggle.value == "(3-7>":
        CYearsNo = 1 #"(3, 7]"
    elif YAC_toggle.value == "(7-10>":
        CYearsNo = 2 #"(7, 10]"
    else:
        CYearsNo = 3 #"(10, 30]"
    
    print("Prawdopodobieństwo odejścia z firmy wynosi: {}".format(df_ds_YAC_Y_N.loc[int(CYearsNo), "Probability_YES"]))


widgets.interact(value_handler, v=YAC_toggle)

interactive(children=(ToggleButtons(description='v', options=('(0-3>', '(3-7>', '(7-10>', '(10-30)'), value='(…

<function __main__.value_handler(v)>

# YearsInCurrentRole vs. Attrition #

In [221]:
YICR = pd.crosstab(df_ds["Attrition"], df_ds["YearsInCurrentRole"]) # Tabela przestawna dla lat przepracowanych na obecnym stanowisku
YICR

YearsInCurrentRole,0,1,2,3,4,5,...,13,14,15,16,17,18
Attrition,,,,,,,,,,,,,
No,171,46,304,119,89,35,...,13,10,6,7,4,2
Yes,73,11,68,16,15,1,...,1,1,2,0,0,0


In [222]:
print(df_ds.groupby(by=["YearsInCurrentRole", "Attrition"]).count()["StockOptionLevel"])

YearsInCurrentRole  Attrition
0                   No           171
                    Yes          73 
1                   No           46 
                    Yes          11 
2                   No           304
                    Yes          68 
3                   No           119
                    Yes          16 
4                   No           89 
                    Yes          15 
5                   No           35 
                    Yes          1  
6                   No           35 
                    Yes          2  
7                   No           191
                    Yes          31 
8                   No           82 
                    Yes          7  
9                   No           61 
                    Yes          6  
10                  No           27 
                    Yes          2  
11                  No           22 
12                  No           9  
                    Yes          1  
13                  No           13 
        

In [223]:
# Create copy of df_ds with Attrition and YearsInCurrentRole only
df_ds_YICR_YES = df_ds.loc[df_ds["Attrition"]=="Yes", ["Attrition", "YearsInCurrentRole"]].copy()
# Gather data in bins to avoid empty values
df_ds_YICR_YES["YearsInCurrentRole_bins"] = pd.cut(df_ds_YICR_YES["YearsInCurrentRole"], bins=[-1,3,7,18])
df_ds_YICR_YES["YearsInCurrentRole_bins"].value_counts()

(-1, 3]    168
(3, 7]     49 
(7, 18]    20 
Name: YearsInCurrentRole_bins, dtype: int64

In [224]:
# Number of employees that left a company vs. the number of years worked at current position
# Create a copy of dataframe with Attrition=Yes
df_ds_YICR_YES = df_ds[df_ds["Attrition"]=="Yes"].copy()
df_ds_YICR_YES["YearsInCurrentRole_bins"] = pd.cut(df_ds_YICR_YES["YearsInCurrentRole"], bins=[-1,3,7,18])
# Count the employees that left the company
df_ds_YICR_YES_only = df_ds_YICR_YES.groupby(by="YearsInCurrentRole_bins").count() ["Attrition"].reset_index()
# Change the name of column Attrition -> Atttrition_YES
df_ds_YICR_YES_only = df_ds_YICR_YES_only.rename(columns = {"Attrition":"Attrition_YES"})
# df_ds_YAC_YES_only

In [225]:
# Number of employees that stayed in a company vs. the number of years worked in a company
# Create a copy of dataframe with Attrition=No
df_ds_YICR_NO = df_ds[df_ds["Attrition"]=="No"].copy()
df_ds_YICR_NO["YearsInCurrentRole_bins"] = pd.cut(df_ds_YICR_NO["YearsInCurrentRole"], bins=[-1,3,7,18])
# Count the employees that stayed in a company
df_ds_YICR_NO_only = df_ds_YICR_NO.groupby(by="YearsInCurrentRole_bins").count() ["Attrition"].reset_index()
# Change the name of column Attrition -> Atttrition_NO
df_ds_YICR_NO_only = df_ds_YICR_NO_only.rename(columns = {"Attrition":"Attrition_NO"})
# df_ds_YAC_NO_only

In [226]:
# Merge tables with trainings and attrition both YES and NO
df_ds_YICR_Y_N = pd.merge(df_ds_YICR_YES_only, df_ds_YICR_NO_only, on="YearsInCurrentRole_bins", how="left")
# Reset index
df_ds_YICR_Y_N.reset_index()
# Add a column with total number of employees
df_ds_YICR_Y_N["Yes+No"] = df_ds_YICR_Y_N["Attrition_YES"] + df_ds_YICR_Y_N["Attrition_NO"]
# Add a column with probability of leaving a company
df_ds_YICR_Y_N["Probability_YES"] = df_ds_YICR_Y_N["Attrition_YES"]/df_ds_YICR_Y_N["Yes+No"]
df_ds_YICR_Y_N

,YearsInCurrentRole_bins,Attrition_YES,Attrition_NO,Yes+No,Probability_YES
0,"(-1, 3]",168,640,808,0.207921
1,"(3, 7]",49,350,399,0.122807
2,"(7, 18]",20,243,263,0.076046


In [227]:
# Characteristics of slider for the number of years worked at current position
from IPython.display import display
from ipywidgets import HBox, Label

YICR_toggle = widgets.ToggleButtons(
    options=['(0-3>', '(3-7>', '(7-18>'],
    description='',
    disabled=False,
    button_style=''
)

# display(YICR_slider)
HBox([Label('Liczba lat przepracowanych na obecnym stanowisku:'), YICR_toggle])

In [228]:
# Nie mogłem się odwołać do tego pogrupowanego indexu, więc zamieniłem na wartości z indexu podstawowego
def value_handler(v):
    if YICR_toggle.value == "(0-3>":
        CYearsNo = 0 #"(-1, 3]"
    elif YICR_toggle.value == "(3-7>":
        CYearsNo = 1 #"(3, 7]"
    else:
        CYearsNo = 2 #"(7, 18]"
    
    print("Prawdopodobieństwo odejścia z firmy wynosi: {}".format(df_ds_YICR_Y_N.loc[int(CYearsNo), "Probability_YES"]))


widgets.interact(value_handler, v=YICR_toggle)

interactive(children=(ToggleButtons(description='v', options=('(0-3>', '(3-7>', '(7-18>'), value='(0-3>'), Out…

<function __main__.value_handler(v)>

# TotalWorkingYears vs. Attrition #

In [229]:
TWY = pd.crosstab(df_ds["Attrition"], df_ds["TotalWorkingYears"]) # Tabela przestawna dla lat przepracowanych na obecnym stanowisku
TWY

TotalWorkingYears,0,1,2,3,4,5,...,34,35,36,37,38,40
Attrition,,,,,,,,,,,,,
No,6,41,22,33,51,72,...,4,3,6,4,1,0
Yes,5,40,9,9,12,16,...,1,0,0,0,0,2


In [230]:
# Settings to display full dataframe date
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 12)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

print(df_ds.groupby(by=["TotalWorkingYears", "Attrition"]).count()["YearsAtCompany"])

TotalWorkingYears  Attrition
0                  No           6 
                   Yes          5 
1                  No           41
                   Yes          40
2                  No           22
                                ..
35                 No           3 
36                 No           6 
37                 No           4 
38                 No           1 
40                 Yes          2 
Name: YearsAtCompany, Length: 71, dtype: int64
<ipython-input-230-8f4ed39b1095>:5: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [231]:
# Create copy of df_ds with Attrition and YearsInCurrentRole only
df_ds_TWY_YES = df_ds.loc[df_ds["Attrition"]=="Yes", ["Attrition", "TotalWorkingYears"]].copy()
# Gather data in bins to avoid empty values
df_ds_TWY_YES["TotalWorkingYears_bins"] = pd.cut(df_ds_TWY_YES["TotalWorkingYears"], bins=[-1,3,7,12,20,40])
df_ds_TWY_YES["TotalWorkingYears_bins"].value_counts()

(3, 7]      68
(7, 12]     63
(-1, 3]     63
(12, 20]    27
(20, 40]    16
Name: TotalWorkingYears_bins, dtype: int64

In [232]:
# Number of employees that left a company vs. the number of years worked at current position
# Create a copy of dataframe with Attrition=Yes
df_ds_TWY_YES = df_ds[df_ds["Attrition"]=="Yes"].copy()
df_ds_TWY_YES["TotalWorkingYears_bins"] = pd.cut(df_ds_TWY_YES["TotalWorkingYears"], bins=[-1,3,7,12,20,40])
# Count the employees that left the company
df_ds_TWY_YES_only = df_ds_TWY_YES.groupby(by="TotalWorkingYears_bins").count() ["Attrition"].reset_index()
# Change the name of column Attrition -> Atttrition_YES
df_ds_TWY_YES_only = df_ds_TWY_YES_only.rename(columns = {"Attrition":"Attrition_YES"})
# df_ds_YAC_YES_only

In [233]:
# Number of employees that stayed in a company vs. the number of years worked in a company
# Create a copy of dataframe with Attrition=No
df_ds_TWY_NO = df_ds[df_ds["Attrition"]=="No"].copy()
df_ds_TWY_NO["TotalWorkingYears_bins"] = pd.cut(df_ds_TWY_NO["TotalWorkingYears"], bins=[-1,3,7,12,20,40])
# Count the employees that stayed in a company
df_ds_TWY_NO_only = df_ds_TWY_NO.groupby(by="TotalWorkingYears_bins").count() ["Attrition"].reset_index()
# Change the name of column Attrition -> Atttrition_NO
df_ds_TWY_NO_only = df_ds_TWY_NO_only.rename(columns = {"Attrition":"Attrition_NO"})
# df_ds_YAC_NO_only

In [234]:
# Merge tables with trainings and attrition both YES and NO
df_ds_TWY_Y_N = pd.merge(df_ds_TWY_YES_only, df_ds_TWY_NO_only, on="TotalWorkingYears_bins", how="left")
# Reset index
df_ds_TWY_Y_N.reset_index()
# Add a column with total number of employees
df_ds_TWY_Y_N["Yes+No"] = df_ds_TWY_Y_N["Attrition_YES"] + df_ds_TWY_Y_N["Attrition_NO"]
# Add a column with probability of leaving a company
df_ds_TWY_Y_N["Probability_YES"] = df_ds_TWY_Y_N["Attrition_YES"]/df_ds_TWY_Y_N["Yes+No"]
df_ds_TWY_Y_N

,TotalWorkingYears_bins,Attrition_YES,Attrition_NO,Yes+No,Probability_YES
0,"(-1, 3]",63,102,165,0.381818
1,"(3, 7]",68,289,357,0.190476
2,"(7, 12]",63,422,485,0.129897
3,"(12, 20]",27,229,256,0.105469
4,"(20, 40]",16,191,207,0.077295


In [235]:
# Characteristics of slider for the number of years worked in total
from IPython.display import display
from ipywidgets import HBox, Label

TWY_toggle = widgets.ToggleButtons(
    options=['(0-3>', '(3-7>', '(7-12>', '(12-20>', '(20-40>'],
    description='',
    disabled=False,
    button_style=''
)

# display(TWS_slider)
HBox([Label('Liczba lat przepracowanych w sumie:'), TWY_toggle])

In [236]:
# Nie mogłem się odwołać do tego pogrupowanego indexu, więc zamieniłem na wartości z indexu podstawowego
def value_handler(v):
    if TWY_toggle.value == "(0-3>":
        CYearsNo = 0 #"(-1, 3]"
    elif TWY_toggle.value == "(3-7>":
        CYearsNo = 1 #"(3, 7]"
    elif TWY_toggle.value == "(7-12>":
        CYearsNo = 2 #"(7, 12]"
    elif TWY_toggle.value == "(12-20>":
        CYearsNo = 3 #"(12, 20]"  
    else:
        CYearsNo = 4 #"(20, 40]"
    
    print("Prawdopodobieństwo odejścia z firmy wynosi: {}".format(df_ds_TWY_Y_N.loc[int(CYearsNo), "Probability_YES"]))


widgets.interact(value_handler, v=TWY_toggle)

interactive(children=(ToggleButtons(description='v', options=('(0-3>', '(3-7>', '(7-12>', '(12-20>', '(20-40>'…

<function __main__.value_handler(v)>

# StockOptionLevel vs. Attrition #

In [237]:
# Pivot table for the level of stock options
SOL = pd.crosstab(df_ds["Attrition"], df_ds["StockOptionLevel"]) 
SOL

StockOptionLevel,0,1,2,3
Attrition,,,,
No,477,540,146,70
Yes,154,56,12,15


In [238]:
# Number of employees that left a company vs. the level of options for stock
# Create a copy of dataframe with Attrition=Yes
df_ds_SOL_YES = df_ds[df_ds["Attrition"]=="Yes"].copy()
# Count the employees that left the company
df_ds_SOL_YES_only = df_ds_SOL_YES.groupby(by="StockOptionLevel").count() ["Attrition"].reset_index()
# Change the name of column Attrition -> Atttrition_YES
df_ds_SOL_YES_only = df_ds_SOL_YES_only.rename(columns = {"Attrition":"Attrition_YES"})
# df_ds_SOL_YES_only

In [239]:
# Number of employees that stayed in a company vs. the level of options for stock
# Create a copy of dataframe with Attrition=No
df_ds_SOL_NO = df_ds[df_ds["Attrition"]=="No"].copy()
# Count the employees that stayed in a company
df_ds_SOL_NO_only = df_ds_SOL_NO.groupby(by="StockOptionLevel").count() ["Attrition"].reset_index()
# Change the name of column Attrition -> Atttrition_NO
df_ds_SOL_NO_only = df_ds_SOL_NO_only.rename(columns = {"Attrition":"Attrition_NO"})
# df_ds_TTLY_NO_only

In [240]:
# Merge tables with trainings and attrition both YES and NO
df_ds_SOL_Y_N = pd.merge(df_ds_SOL_YES_only, df_ds_SOL_NO_only, on="StockOptionLevel", how="left")
# Reset index
df_ds_SOL_Y_N.reset_index()
# Add a column with total number of employees
df_ds_SOL_Y_N["Yes+No"] = df_ds_SOL_Y_N["Attrition_YES"] + df_ds_SOL_Y_N["Attrition_NO"]
# Add a column with probability of leaving a company
df_ds_SOL_Y_N["Probability_YES"] = df_ds_SOL_Y_N["Attrition_YES"]/df_ds_SOL_Y_N["Yes+No"]
print(df_ds_SOL_Y_N)

   StockOptionLevel  Attrition_YES  Attrition_NO  Yes+No  Probability_YES
0  0                 154            477           631     0.244057       
1  1                 56             540           596     0.093960       
2  2                 12             146           158     0.075949       
3  3                 15             70            85      0.176471       


In [243]:
# Characteristics of slider for the level of options for stock

from IPython.display import display
from ipywidgets import HBox, Label

SOL_slider = widgets.IntSlider(
    value=0,
    min=0,
    max=3,
    step=1,
    description='',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

# display(TTLY_slider)
HBox([Label('Poziom opcji na akcje:'), SOL_slider])

In [244]:
# Slider connected with value_handler function returning probability
def value_handler(v):
    TrainNo = SOL_slider.value
    Attr_Y_prob = df_ds_SOL_Y_N.loc[df_ds_SOL_Y_N["StockOptionLevel"]==TrainNo, "Probability_YES"]
    print("Prawdopodobieństwo odejścia z firmy wynosi: {}".format(Attr_Y_prob[TrainNo]))


widgets.interact(value_handler, v=SOL_slider)

interactive(children=(IntSlider(value=3, continuous_update=False, description='v', max=3), Output()), _dom_cla…

<function __main__.value_handler(v)>